In [129]:
import numpy as np
import pandas as pd
from pythainlp.tokenize import word_tokenize
from pythainlp.spell import correct

In [130]:
word_file = pd.read_csv("text_test.csv")

#Negative keyword
neg_pd = pd.read_csv("neg.txt", sep = "/n", header = None)

neg = np.array(neg_pd)

#Neutral keyword
neutral_pd = pd.read_csv("neutral.txt", sep = "/n", header = None)

neutral = np.array(neutral_pd)

#Positive keyword
pos_pd = pd.read_csv("pos.txt", sep = "/n", header = None)

pos = np.array(pos_pd)

In [131]:
word_file_2 = np.array(word_file)
word_file_3 = dict(word_file_2)

print(word_file_3)

{'Mr. Smart': 'สนใจอยากทำ ติดต่อกลับมาด้วยครับ', 'คุณเจมส์': 'สอบถามครัช ค่าบริการอยู่ที่เท่าไหร่', 'Dio': 'Kittanai Puttasitiraj สนมั้ย'}


In [132]:
list_of_list_word = []

for file_sentence in word_file_2:
    token_word = word_tokenize(file_sentence[1])
    list_word = []

    for word in token_word:
        if word.isspace():
            continue
        else:
            new_word = correct(word)
            list_word.append(new_word)

    list_of_list_word.append(list_word)

    
    print(list_of_list_word)


[['สนใจ', 'อยาก', 'ทำ', 'ติดต่อ', 'กลับ', 'ด้วย', 'ครับ']]
[['สนใจ', 'อยาก', 'ทำ', 'ติดต่อ', 'กลับ', 'ด้วย', 'ครับ'], ['สอบถาม', 'คน', 'รัช', 'ค่าบริการ', 'อยู่', 'ที่', 'เท่าไหร่']]
[['สนใจ', 'อยาก', 'ทำ', 'ติดต่อ', 'กลับ', 'ด้วย', 'ครับ'], ['สอบถาม', 'คน', 'รัช', 'ค่าบริการ', 'อยู่', 'ที่', 'เท่าไหร่'], ['Kittanai', 'Puttasitiraj', 'สน', 'มั้ย']]


In [136]:
cus_score = []

for word_order in list_of_list_word:
    score = 0
    for word in word_order:
        if word in pos:
            score += 1
        elif word in neg:
            score -=1
    if score >0:
        status = "Potential"
    else:
        status = "Unpotential"
    cus_score.append(status)

num = 0
while num < len(cus_score):
    word_file_3.update({word_file_2[num][0]:cus_score[num]})
    num += 1

print(word_file_3)

{'Mr. Smart': 'Potential', 'คุณเจมส์': 'Potential', 'Dio': 'Unpotential'}
